In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Diabete

## Lecture des données

In [ ]:
# Directive pour afficher les graphiques dans Jupyter
%matplotlib inline

In [ ]:
# Pandas : librairie de manipulation de données
# NumPy : librairie de calcul scientifique
# MatPlotLib : librairie de visualisation et graphiques
# SeaBorn : librairie de graphiques avancés
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

In [ ]:
# Lecture des données d'apprentissage et de test
d = pd.read_csv("../input/pima-indians-diabetes-database/diabetes.csv")

In [ ]:
d.head()

In [ ]:
sns.pairplot(d, hue="Outcome")

## value_counts

In [ ]:
d.Age.value_counts() 

In [ ]:
d.Age.count()

In [ ]:
d.count()

In [ ]:
d[np.isnan(d.Outcome)].Glucose.value_counts()

In [ ]:
positive = (d.Outcome==1)
negative = (d.Outcome==0)

In [ ]:
d[positive].head()

In [ ]:
d[negative].head()

In [ ]:
d[positive].Age.value_counts()

## quelle est la probabilité diabete/Glucose?

In [ ]:
age = d.Age > 40
classe1 = d.Glucose >= 150
classe2 = d.Glucose < 150
positive = d.Outcome == 1

In [ ]:
diabete = positive & classe1
nondiabete = negative & classe2

In [ ]:
p_diabete = d[diabete & positive].Outcome.count()/d[diabete].Age.count()
print(p_diabete)

In [ ]:
p_nondiabete = d[nondiabete & negative].Age.count()/d[nondiabete].Age.count()
print(p_nondiabete)

In [ ]:
sns.jointplot("Glucose", "Outcome", d, kind='kde');

In [ ]:
sns.jointplot("Age", "Outcome", d, kind='kde');

In [ ]:
fig = sns.FacetGrid(d, hue="Outcome", aspect=3, palette="Set2")
fig.map(sns.kdeplot, "Age", shade=True)
fig.add_legend()

In [ ]:
d.columns

In [ ]:
diabete = d.drop(['SkinThickness'], axis=1)

In [ ]:
diabete.count()

In [ ]:
diabete[['Age','BMI']].describe()

In [ ]:
sns.kdeplot(diabete.BMI, color='blue')
sns.kdeplot(diabete.Age, color='red')

In [ ]:
from sklearn import preprocessing

In [ ]:
minmax = preprocessing.MinMaxScaler(feature_range=(0, 1))
diabete[['Age', 'BMI']] = minmax.fit_transform(diabete[['Age', 'BMI']])

In [ ]:
sns.distplot(diabete.BMI, color='blue')
sns.distplot(diabete.Age, color='red')

In [ ]:
scaler = preprocessing.StandardScaler()
diabete[['Age', 'BMI']] = scaler.fit_transform(diabete[['Age', 'BMI']])

In [ ]:
sns.kdeplot(diabete.BMI, color='blue')
sns.kdeplot(diabete.Age, color='red')

In [ ]:
diabete.info()

In [ ]:
diabete.Outcome = diabete['Outcome'].map({1:'oui', 0:'non'})

In [ ]:
diabete.head()

In [ ]:
diabete.Outcome = diabete['Outcome'].map({"oui":1, "non":0})

In [ ]:
diabete.head()

## Création des jeux d'apprentissage et de test

In [ ]:
X = diabete.drop(['Outcome'], axis=1)
y = diabete.Outcome

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [ ]:
print(X_train.shape)
print(X_test.shape)

## Régression logistique

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
lr = LogisticRegression()
lr.fit(X_train,y_train)

In [ ]:
y_lr = lr.predict(X_test)

## Mesures de performance

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, roc_auc_score,auc, accuracy_score

In [ ]:
print(confusion_matrix(y_test,y_lr))

In [ ]:
print(accuracy_score(y_test,y_lr))

In [ ]:
print(classification_report(y_test, y_lr))

In [ ]:
probas = lr.predict_proba(X_test)

In [ ]:
print(probas)

In [ ]:
dfprobas = pd.DataFrame(probas,columns=['proba_0','proba_1'])
dfprobas['y'] = np.array(y_test)

In [ ]:
dfprobas

In [ ]:
plt.figure(figsize=(10,10))
sns.distplot(1-dfprobas.proba_0[dfprobas.y==0], bins=50)
sns.distplot(dfprobas.proba_1[dfprobas.y==1], bins=50)

In [ ]:
false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test,probas[:, 1])
roc_auc = auc(false_positive_rate, true_positive_rate)
print (roc_auc)

In [ ]:
plt.figure(figsize=(12,12))
plt.title('Receiver Operating Characteristic')
plt.plot(false_positive_rate, true_positive_rate, 'b', label='AUC = %0.2f'% roc_auc)
plt.legend(loc='lower right')
plt.plot([0,1],[0,1],'r--')        # plus mauvaise courbe
plt.plot([0,0,1],[0,1,1],'g:')     # meilleure courbe
plt.xlim([-0.1,1.2])
plt.ylim([-0.1,1.2])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')

## Ajustement des hyperparamètres (Random Forests)

In [ ]:
from sklearn import ensemble
rf = ensemble.RandomForestClassifier()
rf.fit(X_train, y_train)
y_rf = rf.predict(X_test)

In [ ]:
print(classification_report(y_test, y_rf))

In [ ]:
cm = confusion_matrix(y_test, y_rf)
print(cm)

In [ ]:
rf1 = ensemble.RandomForestClassifier(n_estimators=10, min_samples_leaf=10, max_features=3)
rf1.fit(X_train, y_train)
y_rf1 = rf.predict(X_test)
print(classification_report(y_test, y_rf1))

In [ ]:
from sklearn.model_selection import validation_curve
params = np.arange(1, 300,step=30)
train_score, val_score = validation_curve(rf, X, y, 'n_estimators', params, cv=7)
plt.figure(figsize=(12,12))
plt.plot(params, np.median(train_score, 1), color='blue', label='training score')
plt.plot(params, np.median(val_score, 1), color='red', label='validation score')
plt.legend(loc='best')
plt.ylim(0, 1)
plt.xlabel('n_estimators')
plt.ylabel('score');

In [ ]:
from sklearn import model_selection

In [ ]:
param_grid = {
              'n_estimators': [10, 100, 500],
              'min_samples_leaf': [1, 20, 50]
             }
estimator = ensemble.RandomForestClassifier()
rf_gs = model_selection.GridSearchCV(estimator, param_grid)

In [ ]:
rf_gs.fit(X_train, y_train)

In [ ]:
print(rf_gs.best_params_)

In [ ]:
rf2 = rf_gs.best_estimator_

In [ ]:
y_rf2 = rf2.predict(X_test)

In [ ]:
print(classification_report(y_test, y_rf2))

### Importance des caractéristiques

In [ ]:
importances = rf2.feature_importances_
indices = np.argsort(importances)

In [ ]:
plt.figure(figsize=(8,5))
plt.barh(range(len(indices)), importances[indices], color='b', align='center')
plt.yticks(range(len(indices)), X_train.columns[indices])
plt.title('Importance des caracteristiques')